## NAIVE BAYES ALGORİTMASI

### Özel Kütüphane Kullanmadan Naive Bayes Algoritmasını Gerçekleştirme

İlk önce Numpy ve Pandas Kütüphanelerini çalışmamıza ekleyelim.

In [46]:
import numpy as np
import pandas as pd

Ardından Iris veri setimizi Pandas kütüphanesi yardımıyla çalışmamıza ekleyelim ve oluşturduğumuz dataframe'i sadeleştirip karıştıralım.

In [47]:
df = pd.read_csv("Iris.csv")
df.drop("Id",inplace=True,axis=1) # Model için kullanmayacağımız sütunları dataframe'den çıkaralım.
df = df.sample(frac=1) # Dataframe'i karıştıralım.
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
117,7.7,3.8,6.7,2.2,Iris-virginica
85,6.0,3.4,4.5,1.6,Iris-versicolor
124,6.7,3.3,5.7,2.1,Iris-virginica
36,5.5,3.5,1.3,0.2,Iris-setosa
29,4.7,3.2,1.6,0.2,Iris-setosa
...,...,...,...,...,...
106,4.9,2.5,4.5,1.7,Iris-virginica
15,5.7,4.4,1.5,0.4,Iris-setosa
37,4.9,3.1,1.5,0.1,Iris-setosa
108,6.7,2.5,5.8,1.8,Iris-virginica


Verileri "train" ve "test" olarak ayırmak için "split_data" adında bir fonksiyon oluşturalım. Bu fonksiyon parametre olarak çiçeklerin özelliklerini içeren bir "X" numpy dizisi, çiçeklerin hangi sınıflara ait olduklarını içeren bir "y" numpy dizisi ve hangi oranda "train" ve "test" olarak ayırılacaklarına dair "train_size" adında bir yüzdelik değer alır.

In [48]:
def split_data(X, y, train_size):

    start = int(len(X)*train_size)

    X_train = X[:start]
    X_test = X[start:]
    y_train = y[:start]
    y_test = y[start:]

    return X_train, X_test, y_train, y_test

Şimdi sıra En Yakın Naive Bayes Algoritması'nı oluşturmaya geldi. Sırayla KNN sınıfı içinde oluşturduğumuz fonksiyonlara bir göz atalım;

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*5OROQqYWuC6to-5T9OMtXw.jpeg" width="600" height="400">
<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*WAvNX2uhP9eg6P5CkkiL6g.png" width="600" height="400">

calc_prior fonksiyonu bitki verilerinin özelliklerini ve sınıfını argüman olarak alır ve teoremdeki Prior değerini her bir veri için hesaplar.

calc_statistics fonksiyonu bitki verilerinin özelliklerini ve sınıfını argüman olarak alır ve her bir sütunun ortalamasını, varyansını hesaplar ve ardından bu hesapladıklarını numpy dizisine döndürür. Bu hesaplamalar bize gaussian_density fonksiyonu için gerekli hesaplamalardır.

gaussian_density fonksiyonu gaussian yoğunluğunu hesaplar. Belirli bir sınıfa verilen belirli bir hedef değerin olasılığının normal olarak dağıtıldığını varsayacağız.

calc_posterior fonksiyonu ise son olarak her bir veri için posterior değerlerini hesaplar. Ardından bu hesaplanan değerlere göre test verisinin sınıfını tahmin eder. Bu tahminlerini classes adında bir listede geri predict fonksiyonunda çağrıldığı yere döndürür.

In [49]:
class NaiveBayesClassifier():

    def calc_prior(self, features, target):
        '''
        prior probability P(y)
        calculate prior probabilities
        '''
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()

        return self.prior
    
    def calc_statistics(self, features, target):
        '''
        calculate mean, variance for each column and convert to numpy array
        ''' 
        self.mean = features.groupby(target).apply(np.mean, axis=0).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
              
        return self.mean, self.var
    
    def gaussian_density(self, class_idx, x):     
        '''
        calculate probability from gaussian density function (normally distributed)
        we will assume that probability of specific target value given specific class is normally distributed 
        '''
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
#         numerator = np.exp(-((x-mean)**2 / (2 * var)))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    
    def calc_posterior(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for i in range(self.count):
            prior = np.log(self.prior[i]) ## use the log to make it more numerically stable
            conditional = np.sum(np.log(self.gaussian_density(i, x))) # use the log to make it more numerically stable
            posterior = prior + conditional
            posteriors.append(posterior)
        # return class with highest posterior probability
        return self.classes[np.argmax(posteriors)]
     

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        print("Accuracy of Naive Bayes Model: ",accuracy)

Dataframe'de çiçeklerin özelliklerinin bulunduğu sütunları "X" adında bir değişkene, çiçeklerin sınıflarının bulunduğu sütunları ise "y" adında bir değişkene numpy dizisi olarak atayalım. Ardından bu numpy dizilerini yukarıda oluşturduğumuz "split_data" adındaki fonksiyona parametre olarak verelim.

In [50]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = split_data(X, y, train_size=0.7)

In [51]:
# train the model
naiveBayes = NaiveBayesClassifier()
naiveBayes.fit(X_train, y_train)
y_pred = naiveBayes.predict(X_test)
naiveBayes.accuracy(y_test, y_pred)

Accuracy of Naive Bayes Model:  0.9555555555555556


### Sckit-Learn Kütüphanesi Kullanarak KNN Algoritmasını Gerçekleştirme

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [63]:
NB = GaussianNB()
NB.fit(X_train,y_train)
y_predNB = NB.predict(X_test)
print("Accuracy of Naive Bayes Model: ",accuracy_score(y_test,y_predNB))

Accuracy of Naive Bayes Model:  0.9555555555555556
